In [1]:
from __future__ import division
from utils import *
import darknet as dk
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
import os 
import os.path as osp
import pickle as pkl
import pandas as pd
import random
import itertools
import argparse

cuda=True
num_classes = 80
resolution=416
batch_size=2
nms_thresh=0.4
confidence=0.5
images='images/'
det='det/'
link="cfg/yolov3.cfg"
model = dk.Darknet(link)
model.load_weights("weights/yolov3.weights")
classes=load_classes('data/coco.names')

model.net_params['height'] = resolution
inp_dim = int(model.net_params['height'])
assert inp_dim % 32 == 0 
assert inp_dim > 32

model=model.to(get_device())
model.eval()


read_dir=time.time()
try:
    imlist= [osp.join(osp.realpath('.'), images, img) for img in os.listdir(images) if os.path.splitext(img)[1] == '.png'
             or os.path.splitext(img)[1] =='.jpeg' or os.path.splitext(img)[1] =='.jpg']
except NotADirectoryError:
    imlist=[]
    imlist.append(osp.join(osp.realpath('.'),images))
except FileNotFoundError:
    print('no file found')

if not os.path.exists(det):
    os.makedirs(det)

load_batch=time.time()
#loaded_ims=[cv2.imread(x) for x in imlist]
#loaded_ims=[letterbox_image(cv2.imread(x),[inp_dim for x in range(len(imlist))]) for x in loaded_ims]
#loaded_ims=list(map(letterbox_image,loaded_ims,[inp_dim for x in range(len(imlist))]))
loaded_imgs = [cv2.imread(x) for x in imlist]

batches=list(map(prep_image,imlist,[inp_dim for x in range(len(imlist))]))

img_batches=[x[0] for x in batches]
orig_imgs=[x[1] for x in batches]
img_dim_list=[x[2] for x in batches]
img_dim_list=torch.FloatTensor(img_dim_list).repeat(1,2)
img_dim_list=img_dim_list.to(get_device())


leftover=0
if(len(img_dim_list)%batch_size):
    leftover=1
    
if batch_size!=1:
    num_batches=len(imlist)//batch_size+leftover
    img_batches=[torch.cat((img_batches[i*batch_size:min((i+1)*batch_size,len(img_batches))])) for i in range(num_batches)]
    
i=0
write=False

start_det_loop=time.time()
objs={}

for batch in img_batches:
    start=time.time()
    
    batch=batch.to(get_device())
    
    with torch.no_grad():
        #model=model.to(get_device())
        prediction=model(Variable(batch))
    #print(prediction) 
    prediction=write_results(prediction,confidence,num_classes,nms_thresh)
    
    if type(prediction)==int:
        i+=1
        continue
        
    end=time.time()
    
    prediction[:,0]+=i*batch_size
    
    if not write:
        output=prediction
        write=1
    else:
        output=torch.cat((output,prediction))
    
    for img_num,image in enumerate(imlist[i*batch_size:min((i+1)*batch_size,len(imlist))]):
        img_id=i*batch_size+img_num
        objs=[classes[int(x[-1])] for x in output if int(x[0])==img_id]
        
        print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
        print("{0:20s} {1:s}".format("Objects Detected:", " ".join(objs)))
        print("----------------------------------------------------------")
        
    i += 1
    if cuda:
        torch.cuda.synchronize()
try:
    output
except NameError:
    print('no detections')
        
    
img_dim_list=torch.index_select(img_dim_list,0,output[:,0].long())
scaling_factor=torch.min(inp_dim/img_dim_list,1)[0].view(-1,1)
    
output[:,[1,3]]-=(inp_dim-scaling_factor*img_dim_list[:,0].view(-1,1))/2
output[:,[2,4]]-=(inp_dim-scaling_factor*img_dim_list[:,1].view(-1,1))/2
    
output[:,1:5] /= scaling_factor
    
for i in range(output.shape[0]):
    output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, img_dim_list[i,0])
    output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, img_dim_list[i,1])
        
output_recast = time.time()
class_load = time.time()
colors=pkl.load(open('pallete','rb'))
    
draw=time.time()
    

    
list(map(lambda x: apply_box(x,orig_imgs,classes,colors),output))

det_names = pd.Series(imlist).apply(lambda x: "{}/det_{}".format(det,x.split("/")[-1]))
    
list(map(cv2.imwrite, det_names, orig_imgs))
    
end = time.time()
    
print()
print("SUMMARY")
print("----------------------------------------------------------")
print("{:25s}: {}".format("Task", "Time Taken (in seconds)"))
print()
print("{:25s}: {:2.3f}".format("Reading addresses", load_batch - read_dir))
print("{:25s}: {:2.3f}".format("Loading batch", start_det_loop - load_batch))
print("{:25s}: {:2.3f}".format("Detection (" + str(len(imlist)) +  " images)", output_recast - start_det_loop))
print("{:25s}: {:2.3f}".format("Output Processing", class_load - output_recast))
print("{:25s}: {:2.3f}".format("Drawing Boxes", end - draw))
print("{:25s}: {:2.3f}".format("Average time_per_img", (end - load_batch)/len(imlist)))
print("----------------------------------------------------------")

    
torch.cuda.empty_cache()

dog-cycle-car.png    predicted in  0.417 seconds
Objects Detected:    bicycle truck dog
----------------------------------------------------------
dog.jpg              predicted in  0.417 seconds
Objects Detected:    bicycle truck dog
----------------------------------------------------------
eagle.jpg            predicted in  0.019 seconds
Objects Detected:    bird
----------------------------------------------------------
giraffe.jpg          predicted in  0.019 seconds
Objects Detected:    zebra giraffe giraffe
----------------------------------------------------------
herd_of_horses.jpg   predicted in  0.020 seconds
Objects Detected:    horse horse horse horse
----------------------------------------------------------
img1.jpg             predicted in  0.020 seconds
Objects Detected:    person dog
----------------------------------------------------------
img2.jpg             predicted in  0.023 seconds
Objects Detected:    train
----------------------------------------------------